<a href="https://colab.research.google.com/github/weymouth/MarineHydro/blob/master/notebooks/panel_method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2D Potential flow panel method 🚢

A panel method is a numerical approach to constructing potential flows around engineering shapes such as a wing or ship hull. Panel methods used in industry are typically 3D, but we will use a 2D approach which is sufficient for strip theory calculations.

## Potential and velocity of panel source 🚰

The 2D vesion of a panel is a line _segment_. In general this segment will run from some point $(x_i,y_i)$ to $(x_{i+1},y_{i+1})$ and will have a potential flow singularity distributed over it. 

For example, a 2D source potential is proportional to

$$ G(x,y,x_s,y_x) = \log(r_s) = \frac 12 \log(r_s^2) $$

where $r_s^2 = (x-x_s)^2+(y-y_s)^2$ is the square-distance from the point $(x_s,y_s)$ on the segment to a point $(x,y)$ in space. $G$ is called a [Green's function](https://en.wikipedia.org/wiki/Green%27s_function) and you can think of it as the building block for the panel method. 

The potential of the panel is the integrated superposition of $G$ along the segment, ie

$$ \phi_i(x,y) = q_i \int_{s_i}^{s_{i+1}} G(x,y,x_s,y_s) ds = q_i F_i^\phi(x,y)$$

where $q_i$ is the source strength per segment length. We will assume the strength is constant along each panel, allowing us to pull it out of the integral. The function $F_i^\phi$ is the potential **influence function**, the potential induced by segment $i$ per unit strength.

The potential flow velocity is the gradient of this potential (as usual) and since the $q_i$ is a constant, we can pull it outside the derivative

$$ \vec u_i = \vec\nabla\phi_i = q_i\vec\nabla F^\phi_i=q_iF^{\vec u_i}\ ,\quad F^{\vec u_i} = \left[\frac{\partial F^\phi_i}{\partial x},\frac{\partial F^\phi_i}{\partial y}\right]$$

which defines the influence functions for the velocity $\vec u$.

We will approximate the integral with a [Gaussian-quadrature](https://en.wikipedia.org/wiki/Gaussian_quadrature#/media/File:Comparison_Gaussquad_trapezoidal.svg) and the derivatives with [finite differences](https://en.wikipedia.org/wiki/Finite_difference#/media/File:Finite_difference_method.svg).

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

def source(x, y, xs, ys): return 0.5 * np.log((x-xs)**2 + (y-ys)**2)

GAUSS2 = 0.5 * (1 + np.sqrt(1/3))  # gaussian-quadrature sample point
def potential(x, y, x0, y0, x1, y1, G=source, args=()):
    "Gaussian quadrature estimate of the potential influence function"
    def dG(s): return G(x, y, x0*(1-s)+x1*s, y0*(1-s)+y1*s, *args)
    h = np.sqrt((x1-x0)**2 + (y1-y0)**2)
    return 0.5*h*sum(dG(s) for s in [GAUSS2, 1-GAUSS2])

def velocity(x, y, x0, y0, x1, y1, G=source, args=(), eps=1e-6):
    """Finite difference estimate of the velocity influence function"""
    def phi(x, y): return potential(x, y, x0, y0, x1, y1, G, args)
    return np.array(((phi(x+eps, y) - phi(x-eps, y)) / (2*eps),   # dphi/dx
                     (phi(x, y+eps) - phi(x, y-eps)) / (2*eps)))  # dphi/dy

# Example segment
x0, y0 = -0.1, -0.5
x1, y1 = 0.4, 0.17

# Visualize the flow
XY = np.meshgrid(np.linspace(-2, 2, 22), np.linspace(-2, 2, 22))
UV = velocity(*XY, x0, y0, x1, y1)
Phi = potential(*XY, x0, y0, x1, y1)

# Plot it up
plt.figure(figsize=(7, 7))
plt.quiver(*XY, *UV)
plt.contour(*XY, Phi, 20)
plt.plot([x0,x1], [y0,y1], c='b')
plt.axis('equal');

The resulting flow is the same as a source flow when $r \gg ds$. However, the velocity is finite and smooth near the panel. This "well behaved" velocity makes distributed potentials a good choice for numerical methods.

> **Note:** Using numerical approximations makes our `potential` and `velocity` functions **re-usable**. We have used a source singularity, but we could use some other Green's function if we want. _Different singularities are good for building different types of flows_. [This interactive webapp](https://weymouth.github.io/fluid-dynamics/vortexpanel/index.html) uses vortex segments to predict the flow and forces over a thin wing, and at the bottom of this notebook we will use a wave singularity for free surface flows.

---

## Superposition 💧➕💧➕💧= 💦

The next step is to link multiple panels together to form a geometry. The total potential and velocity will simply be the superposition (sum) of each 
of source panels given above

$$\phi=\sum_i \phi_i=\sum_i q_i F_i^\phi\ ,\quad \vec u = \sum_i \vec u_i=\sum_i q_i F_i^{\vec u}.$$ 

Let's define a set of $N$ panels by connecting $N+1$ points $x_0\ldots x_N$ and $y_0\ldots y_N$. Then the Python code for the potential or velocity is a simply `sum` over `i in range(N-1)`. 

Here's a superposition version of the plotting code above, and a test on a few panels.

In [ ]:
def plot_flow(x, y, q, XY, G=source, args=(), size=(7,7), ymax=None):
    # Loop through segments, superimposing the velocity
    def uv(i): return q[i]*velocity(*XY, x[i], y[i], x[i+1], y[i+1], G, args)
    UV = sum(uv(i) for i in range(len(x)-1))

    # Create plot
    plt.figure(figsize=size)
    ax = plt.axes()
    ax.set_aspect('equal', adjustable='box')

    # Plot vectors and segments
    Q = plt.quiver(*XY, *np.real(UV))
    if np.iscomplexobj(UV):
        Q._init()
        plt.quiver(*XY, *np.imag(UV), scale=Q.scale, color='g')
    plt.plot(x, y, c='b')
    plt.ylim(None, ymax)

plot_flow([-1,1,1,-1], [-1,-1,1,1], [1,-1,1], XY)

Excellent! We can now plot the flow induced by a set of panels. Move around the points and change the strength `q` to see how the flow changes.

> **Note:** The velocity isn't _that_ smooth since we've only approximated the integral with two samples (you can see where they are). This will be less of a problem when we use smaller panels. Lets define a function to create `N` panels in the shape of an ellipse.

In [ ]:
def ellipse(N, a=1, b=1, theta1=np.pi):
    "x,y arrays around an elliptical arc"
    theta = np.linspace(-np.pi, theta1, N+1)  # N+1 points for N panels
    return a*np.cos(theta), b*np.sin(theta)

def mask_grid(x, y, mask):
    "delete mesh points where mask is true"
    def delete(a): return np.ma.masked_array(a, mask(x,y)).compressed()
    return np.array((delete(x), delete(y)))

# geometry
N, rx, ry = 32, 1.5, 0.75
x, y = ellipse(N, rx, ry)
EXY = mask_grid(*XY, mask = lambda x, y: (x/rx)**2 + (y/ry)**2<1)

q = np.ones_like(x)  # we don't know q yet!
plot_flow(x, y, q, EXY)



## Linear system 🚣 ✖ ⚡ = 🚤

The figure above assumes $q_i=1$, but we can get different flows around the same geometry by adjusting this strength array. But how can we set $q_i$ to, say, compute the flow over an ellipse moving to the left?

The Lapace equation $\nabla^2 \phi = 0$ is satisfied automatically for any set of panel strengths so we need to determine $q_i$ another way. 

Instead the strength is determined using the normal velocity body boundary condition 
$$\sum_i \vec u_i\cdot \hat n = \vec U \cdot \hat n \quad\text{on the body}$$ 
where $\hat n$ is the body normal and $\vec U$ is the body velocity. 

This equation can be applied at every panel $j$. Substitution gives

$$\sum_i q_i F^{\vec u}_i(x_j,y_j)\cdot \hat n_j = \sum_i a_{ji} q_i  = \vec U \cdot \hat n_j \quad\text{on the body}$$

where $a_{ji}$ is the normal velocity influence of panel $i$ on panel $j$. 

This is a system of linear equations which we can write in the familiar matrix form:
$$ \mathbf{A q} = \mathbf{b}$$
where $b_j=\vec U \cdot \hat n_j$ and bold face indicates matrices and vectors. This linear system of equations can be solved for $q_i$ using [`numpy.linalg.solve`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.solve.html).

> **Note:** The velocity induced by a panel on itself ($a_{ii}$) isn't accurately computed using the `velocity` influence function, so I've added this contribution manually using $\mathbf{A}=\mathbf{A}+\pi\mathbf{I}$, where $\mathbf{I}$ is the identity matrix.

In [ ]:
def properties(x0, y0, x1, y1):
    "properties of a line segment"
    sx, sy = x1-x0, y1-y0          # segment vector
    xc, yc = x0+0.5*sx, y0+0.5*sy  # segment center
    h = np.sqrt(sx**2 + sy**2)     # segment length
    nx, ny = sy/h, -sx/h           # segment unit normal
    return xc, yc, nx, ny, h

def construct_A(x, y, G=source, args=(), aii=np.pi):
    "construct the velocity influence matrix"
    # influence of panel i on the normal velocity at each panel center
    xc, yc, nx, ny, _ = properties(x[:-1], y[:-1], x[1:], y[1:])

    def u_n(i):
        u, v = velocity(xc, yc, x[i], y[i], x[i+1], y[i+1], G, args)
        return u*nx + v*ny

    # construct matrix
    A = np.array([u_n(i) for i in range(len(yc))]).T
    A += aii * np.eye(len(yc))  # add panel self-influence
    return A, nx, ny

Ux, Uy = -1, 1  # change the motion direction with this
x, y = ellipse(N, rx, ry)
A, nx, ny = construct_A(x, y)
q = np.linalg.solve(A, Ux*nx + Uy*ny)
plot_flow(x, y, q, EXY)

Awesome! We can now solve for the potential flow around any set of moving panels!! **But is this flow correct?!?**

> **Note:** We should never trust a numerical solution until we have tested the solver. 

At the very least, we insist that the solution $q$ **converges** as we increase the number of panels. This is easy to check:

In [ ]:
for N in 2**(np.arange(6)+2):
    x, y = ellipse(N, rx, ry)
    A, nx, ny = construct_A(x, y)
    q = np.linalg.solve(A, Ux*nx + Uy*ny)
    plt.plot(np.linspace(-np.pi, np.pi, N), q, label=N)
plt.xlabel(r'$\theta$', size=14)
plt.ylabel('q', rotation=0, size=14)
plt.legend(title='N');

We can see the solution does converge with $N$ and the difference between using 64 and 128 panels is extremely small. 

---

## Added mass 🐙

One of the most important applications of potential flow is the estimation of added mass due to body oscillations.  The oscillation displacement is written as

$$\vec \xi = |\xi_k| \cos(\omega t) \hat e_k $$
where $|\xi_k|,\omega$ are the oscillation amplitude and frequency and $k=1,2$ indicates motion in the $\hat x = \hat e_1$ and $\hat y = \hat e_2$ directions.

There are two steps to predicting the added mass: 

**1. Determine the the potential for motion in direction $k$**

The body velocity $\frac{\partial}{\partial t}{\vec\xi}$ determines the potential via the $u_n=U_n$ boundary condition. If we define the unsteady potential as

$$\phi(t) = -|\phi_k||\xi_k|\omega\sin(\omega t) $$
where $|\phi_k|$ is the potential _per motion velocity_, then substituting the potential and body velocity into the boundary condition gives

$$ -\omega\sin(\omega t) |\xi_k| \left(\frac{\partial |\phi_k|}{\partial n} -n_k\right) = 0.$$
Since the sine isn't always zero, the term in parenthesis must be set to zero. 

> **Note:** This is the same body boundary condition we used in the steady case. 

Therefore, defining $|\phi_k| = \sum_i q_{k,i} F_i^\phi$ we can use the same normal velocity influence matrix to solve for the source strength of each panel due to body oscillations in direction $k$ as
$$ \mathbf{A}\mathbf{q}_k = \mathbf{n}_k$$

**2. Integrate the pressure to determine the force**

The added mass force is the integrated pressure due 
to the unsteady potential

$$ \vec f = \rho \oint \frac{\partial\phi}{\partial t} \hat n ds$$

Substiting $\phi$ gives the $l$-component of the added mass force as

$$ f_l = \sum_k \rho \oint -\omega^2\cos(\omega t)|\phi_k||\xi_k|\ n_l\ ds = -\sum_{k} \ddot\xi_k m_{kl}$$
where $\ddot\xi_k=-\omega^2 |\xi_k|\cos(\omega t)$ is the body acceleration and $m_{kl}$ is the added mass matrix
$$ m_{kl} = -\rho\oint |\phi_k| n_l ds = -\rho\sum_j |\phi_{k,j}| n_{l,j} h_j . $$

Substituting the equation for $|\phi_k|$ gives
$$ m_{kl} = -\rho\sum_j\sum_i q_{k,i} F_i^\phi(x_j,y_j) n_{l,j} h_j $$

That's it. Solve for each $q_k$ as before, and then plug it into double summation.

Newman gives the analytic the added mass of an ellipse with semi-axis lengths $r_x,r_y$ as $m_{11} = \rho \pi r_y^2,\ m_{22}=\rho\pi r_x^2$. Therefore, we can use this case to **validate** our numerical method!

> **Note:** A numerical method is validated if the solution converges to a known exact solution. 

In [ ]:
def added_mass(x, y, G=source, args=(), rho=1):
    "Compute the added mass matrix"
    # strength due to x,y motion
    A, n1, n2 = construct_A(x, y, G, args)
    q1 = np.linalg.solve(A, n1)
    q2 = np.linalg.solve(A, n2)

    # potential due to y,z motion (times panel width)
    xc, yc, _, _, h = properties(x[:-1], y[:-1], x[1:], y[1:])
    hF = [h*potential(xc, yc, x[i], y[i], x[i+1], y[i+1], G, args) for i in range(len(yc))]
    phi1, phi2 = hF@q1, hF@q2  # potential influence matrix times strength

    # sum over panels
    return -rho * np.matrix([[phi1@n1,phi1@n2], [phi2@n1,phi2@n2]])

# test add_mass
def err(m, r): return 100*(m/(np.pi*r**2)-1)
print('    |   % error\n N  |  m11 |  m22 ')
rx, ry = 1.5, 0.75
for N in 2**(np.arange(6)+4):
    m = added_mass(*ellipse(N, rx, ry))
    print(f'{N:3} | {err(m[0,0],ry):.2f} | {err(m[1,1],rx):.2f}')

The error is less than 1% for the added mass coefficients using only 32-64 panels and converges to zero with more panels. The code is validated!

> **Note:** The error goes down by $\times 2$ when the number of panels increases by $\times 2$. Therefore the error is proportional to the panel size, $\varepsilon\sim h$, which is called _linear_ convergence. 